<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/code/VGG16_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 모델 형성

In [ ]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 16.3 MB/s 


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import datetime
import time
import tempfile

from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
import tensorflow_model_optimization as tfmot

In [ ]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

"""
print(raw_train_x.shape)
print(raw_test_x.shape)
print(raw_train_y.shape)
print(raw_test_y.shape)
"""

'\nprint(raw_train_x.shape)\nprint(raw_test_x.shape)\nprint(raw_train_y.shape)\nprint(raw_test_y.shape)\n'

In [ ]:
# float을 넣을거면 0~1 사이 값으로 바꿔야함
# integer 형태로 넣어보기
train_x = raw_train_x[:45000].astype(np.float32)/255.0
valid_x = raw_train_x[45000:].astype(np.float32)/255.0
test_x = raw_test_x.astype(np.float32)/255.0


train_y = raw_train_y[:45000]
valid_y = raw_train_y[45000:]
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


"""
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)
print(train_y.shape)
print(valid_y.shape)
print(test_y.shape)

def show_sample(i):
  print(raw_train_y[i][0], labels[raw_train_y[i][0]])
  plt.imshow(raw_train_x[i])
  plt.show()

for i in [2, 10, 12, 14]:
  show_sample(i)
"""

'\nprint(train_x.shape)\nprint(valid_x.shape)\nprint(test_x.shape)\nprint(train_y.shape)\nprint(valid_y.shape)\nprint(test_y.shape)\n\ndef show_sample(i):\n  print(raw_train_y[i][0], labels[raw_train_y[i][0]])\n  plt.imshow(raw_train_x[i])\n  plt.show()\n\nfor i in [2, 10, 12, 14]:\n  show_sample(i)\n'

In [ ]:
vgg = VGG16(  #weights = 'imagenet',  #그냥 초기화 하는거면, 이거 지우기
                            include_top = False,
                            input_shape=(32,32,3)
                            )

# vgg conv 구조만 사용하고 마지막 FC layer는 다른거 사용

fc_layer = keras.Sequential([
                             layers.Flatten(),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(10, activation = "sigmoid")
                             ])

model = keras.Sequential([vgg,
                          fc_layer
                          ])
model.summary()
# vgg.summary()
# fc_layer.summary()

58900480/58889256 [==============================] - 94s 2us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
sequential (Sequential)      (None, 10)                2634762   
Total params: 17,349,450
Trainable params: 17,349,450
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Callback 함수 지정 해주기
early_stop = EarlyStopping(patience=30) 
mc = ModelCheckpoint("./best_model/vgg_original_checkpoint", 
                     save_best_only=True,
                     monitor = 'val_loss',
                     verbose = 1,
                     mode = 'min') 
reduce_lr  = ReduceLROnPlateau(monitor = 'val_loss',
                               factor=0.5, 
                               patience=5
                               ) 
# tensorboard 관련 
def make_Tensorboard_dir(dir_name):
  root_logdir = os.path.join(os.curdir, dir_name)
  sub_dir_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  return os.path.join(root_logdir, sub_dir_name)

dir_name = "Learning_log"
TB_log_dir = make_Tensorboard_dir(dir_name)
TensorB = TensorBoard(log_dir = TB_log_dir)


#optimizaer  조정 해주기
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])



# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y,
          epochs=100,
          verbose=1,
          batch_size=32,
          #validation_split = 0.1
          validation_data = (valid_x, valid_y),
          callbacks = [early_stop, reduce_lr , mc]
          )

Epoch 1/100
1407/1407 [==============================] - 265s 178ms/step - loss: 0.9211 - accuracy: 0.6831 - val_loss: 0.7242 - val_accuracy: 0.7516

Epoch 00001: val_loss improved from inf to 0.72417, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoint\assets
Epoch 2/100
1407/1407 [==============================] - 251s 178ms/step - loss: 0.5360 - accuracy: 0.8228 - val_loss: 0.5454 - val_accuracy: 0.8218

Epoch 00002: val_loss improved from 0.72417 to 0.54544, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoint\assets
Epoch 3/100
1407/1407 [==============================] - 251s 178ms/step - loss: 0.3863 - accuracy: 0.8714 - val_loss: 0.5271 - val_accuracy: 0.8338

Epoch 00003: val_loss improved from 0.54544 to 0.52712, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoi

In [ ]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=1)

print(predicted)

## 75.2
## 67.4

313/313 [==============================] - 16s 50ms/step - loss: 1.5378 - accuracy: 0.87415s - loss: 1.524 - ETA: 4s - l
loss= 1.5377521514892578
acc= 0.8741000294685364
[5 8 8 ... 5 1 7]


In [ ]:
model.save("./saved_model/vgg_original.h5")
model.save("./saved_model/vgg_original_checkpoint")

INFO:tensorflow:Assets written to: ./saved_model/vgg_original_checkpoint\assets


## int_8 Quantization 진행

In [ ]:
# 구조까지 들어가 있는거
model = tf.keras.models.load_model('./best_model/vgg_original_checkpoint')

In [ ]:
# input 에 대해서 변수 설정
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_x).batch(1).take(100):
    yield [input_value]

In [ ]:
# int 8 로 quantization 진행하기
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmp6jj7wvkm\assets


INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmp6jj7wvkm\assets


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# 그냥 파일 형태만 tflite로 변환 (float 형태임)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
# 모델 저장 해주는 과정
# 현재는 저장이 불가한 상태로 나옴

import pathlib

tflite_models_dir = pathlib.Path("./tflite/vgg_tflite/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)


In [ ]:

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"vgg_original_tflite.tflite"
#tflite_model_file.write_bytes(tflite_model)
open(tflite_model_file, "wb") .write(tflite_model)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"vgg_quantization_tflite.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

17486184

In [ ]:
def run_tflite_model(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [ ]:
def evaluate_model(tflite_file, model_type):
  global test_x
  global test_y

  test_image_indices = range(test_x.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_y.reshape(-1)== predictions) * 100) / len(test_x)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_x)))

In [ ]:
evaluate_model(tflite_model_file, model_type="Float")

Float model accuracy is 86.2100% (Number of test samples=10000)


In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

KeyboardInterrupt: 

In [ ]:
def run_tflite_time(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  fps = np.zeros((len(test_image_indices),))
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

        
    start = time.time()
    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    end = time.time()
    fps[i] = (end - start)

  return fps

In [ ]:
# Change this to test a different image
def test_model_time(tflite_file, model_type):
  global test_x
  test_image_indices = range(test_x.shape[0])
  fps = run_tflite_time(tflite_file, test_image_indices)
  print(np.mean(fps))
  print(1 / np.mean(fps))

    

In [ ]:
test_model_time(tflite_model_file, model_type="Float")

0.012972253274917602
77.08760990147657


In [ ]:
test_model_time(tflite_model_quant_file,  model_type="Quantized")

KeyboardInterrupt: 

# Pruning 진행

## Sparsity = 0.2

In [ ]:
model = tf.keras.models.load_model('./best_model/vgg_original_checkpoint')
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 1
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.2,
                                                               begin_step=0,
                                                               end_step=-1,
                                                               frequency = 100
                                                               )
}

vgg_pruning_pruning_2 = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
vgg_pruning_pruning_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

vgg_pruning_pruning_2.summary()

Baseline test accuracy: 0.8621000051498413
Saved baseline model to: C:\Users\yutan\AppData\Local\Temp\tmpp67qxg2c.h5


In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

vgg_pruning_pruning_2.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

C:\Users\yutan\anaconda3\envs\tensorflow\lib\site-packages\keras\backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


   6/1266 [..............................] - ETA: 6:10 - loss: 11.2548 - accuracy: 0.3490WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0638s vs `on_train_batch_end` time: 0.2077s). Check your callbacks.


1266/1266 [==============================] - 252s 195ms/step - loss: 1.7820 - accuracy: 0.2993 - val_loss: 1.2011 - val_accuracy: 0.5671


In [ ]:
_, model_for_pruning_accuracy = vgg_pruning_pruning_2.evaluate(
   test_x, test_y, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
vgg_pruning_pruning_2_eport = tfmot.sparsity.keras.strip_pruning(vgg_pruning_pruning_2)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(vgg_pruning_pruning_2_eport)
vgg_pruning_pruning_2_tflite = converter.convert()

import pathlib

tflite_models_dir = pathlib.Path("./tflite/vgg_tflite/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)



vgg_pruning_pruning_2_file = tflite_models_dir/"vgg_pruning_pruning_2_tflite.tflite"
#tflite_model_file.write_bytes(tflite_model)
open(vgg_pruning_pruning_2_file, "wb") .write(vgg_pruning_pruning_2_tflite)


INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmpmhk6w5xh\assets


INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmpmhk6w5xh\assets


69409072

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
get_gzipped_model_size(vgg_pruning_pruning_2_file)
get_gzipped_model_size(tflite_model_quant_file)

NameError: name 'tflite_model_quant_file' is not defined